# Jupyter Notebooks for Handwritten Digit Classification
### Author - [Deepayan Thakur](https://github.com/Deepayan-Thakur)

This directory contains the Jupyter notebooks used for various stages of the Handwritten Digit Classification project.


In [4]:
# Part 1: Training the Model
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
# Load and preprocess MNIST dataset
def load_and_preprocess_mnist():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    
    # Normalize pixel values
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
    
    # Reshape for CNN
    x_train = x_train.reshape(-1, 28, 28, 1)
    x_test = x_test.reshape(-1, 28, 28, 1)
    
    return x_train, y_train, x_test, y_test


In [6]:
# Create CNN model
def create_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    
    return model


In [7]:
# Train model
def train_digit_classifier():
    x_train, y_train, x_test, y_test = load_and_preprocess_mnist()
    model = create_model()
    
    model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))
    model.save('digit_classifier.h5')
    
    return model


In [8]:
# Part 2: Real-time Detection with OpenCV
def preprocess_image(img):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply thresholding
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    # Resize to 28x28
    resized = cv2.resize(thresh, (28, 28), interpolation=cv2.INTER_AREA)
    # Normalize and reshape
    normalized = resized.astype('float32') / 255.0
    preprocessed = normalized.reshape(1, 28, 28, 1)
    return preprocessed

In [9]:
def real_time_detection():
    # Load the trained model
    model = tf.keras.models.load_model('digit_classifier.h5')
    
    # Start video capture
    cap = cv2.VideoCapture(0)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        # Define region of interest (ROI)
        roi = frame[100:400, 100:400]
        cv2.rectangle(frame, (100, 100), (400, 400), (0, 255, 0), 2)
        
        # Process ROI
        processed = preprocess_image(roi)
        
        # Make prediction
        prediction = model.predict(processed)
        predicted_digit = np.argmax(prediction[0])
        confidence = prediction[0][predicted_digit]
        
        # Display results
        cv2.putText(frame, f'Digit: {predicted_digit}', 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                    1.0, (0, 255, 0), 2)
        cv2.putText(frame, f'Confidence: {confidence:.2f}', 
                    (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 
                    1.0, (0, 255, 0), 2)
        
        cv2.imshow('Digit Detection', frame)
        
        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()


In [10]:
if __name__ == "__main__":
    # First train the model (only need to do this once)
    print("Training model...")
    model = train_digit_classifier()
    
    # Then run real-time detection
    print("Starting real-time detection...")
    real_time_detection()

Training model...


c:\Users\Deepayan Thakur\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 27s 12ms/step - accuracy: 0.8989 - loss: 0.3263 - val_accuracy: 0.9853 - val_loss: 0.0422
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.9844 - loss: 0.0491 - val_accuracy: 0.9883 - val_loss: 0.0347
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9894 - loss: 0.0347 - val_accuracy: 0.9913 - val_loss: 0.0280
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.9915 - loss: 0.0253 - val_accuracy: 0.9901 - val_loss: 0.0313
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.9939 - loss: 0.0191 - val_accuracy: 0.9922 - val_loss: 0.0229


Starting real-time detection...


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━

In [11]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

def plot_training_history(history):
    # Create a figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot accuracy
    ax1.plot(history.history['accuracy'], label='Training Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax1.set_title('Model Accuracy over Epochs')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True)
    
    # Plot loss
    ax2.plot(history.history['loss'], label='Training Loss')
    ax2.plot(history.history['val_loss'], label='Validation Loss')
    ax2.set_title('Model Loss over Epochs')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()

def create_confusion_matrix(model, x_test, y_test):
    # Get predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    # Calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred_classes)
    
    # Create heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig('confusion_matrix.png')
    plt.close()
    
    # Print classification report
    report = classification_report(y_test, y_pred_classes)
    print("\nClassification Report:")
    print(report)
    
    return cm, report

def analyze_model_performance():
    # Load and preprocess data
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
    
    x_train = x_train.reshape(-1, 28, 28, 1)
    x_test = x_test.reshape(-1, 28, 28, 1)
    
    # Create and train model
    model = create_model()  # Using the function from previous artifact
    history = model.fit(x_train, y_train, 
                       epochs=5, 
                       validation_data=(x_test, y_test),
                       verbose=1)
    
    # Plot training history
    plot_training_history(history)
    
    # Create confusion matrix and get performance metrics
    cm, report = create_confusion_matrix(model, x_test, y_test)
    
    # Plot sample predictions
    plot_sample_predictions(model, x_test, y_test)
    
    return history, cm, report

def plot_sample_predictions(model, x_test, y_test, num_samples=10):
    # Get random samples
    indices = np.random.randint(0, x_test.shape[0], num_samples)
    sample_images = x_test[indices]
    true_labels = y_test[indices]
    
    # Get predictions
    predictions = model.predict(sample_images)
    pred_labels = np.argmax(predictions, axis=1)
    
    # Plot samples
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    axes = axes.ravel()
    
    for idx, ax in enumerate(axes):
        ax.imshow(sample_images[idx].reshape(28, 28), cmap='gray')
        ax.axis('off')
        color = 'green' if pred_labels[idx] == true_labels[idx] else 'red'
        ax.set_title(f'Pred: {pred_labels[idx]}\nTrue: {true_labels[idx]}', 
                    color=color)
    
    plt.tight_layout()
    plt.savefig('sample_predictions.png')
    plt.close()

if __name__ == "__main__":
    # Run the analysis
    history, confusion_matrix, classification_report = analyze_model_performance()

Epoch 1/5


c:\Users\Deepayan Thakur\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - accuracy: 0.8935 - loss: 0.3370 - val_accuracy: 0.9866 - val_loss: 0.0460
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9836 - loss: 0.0515 - val_accuracy: 0.9894 - val_loss: 0.0359
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - accuracy: 0.9893 - loss: 0.0344 - val_accuracy: 0.9896 - val_loss: 0.0320
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.9915 - loss: 0.0262 - val_accuracy: 0.9902 - val_loss: 0.0330
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - accuracy: 0.9938 - loss: 0.0195 - val_accuracy: 0.9901 - val_loss: 0.0338
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       980
           1       0.99      1.00      0.99      1135
           2       0.99      0.99      0.99      1032
           3       0.98      1.00      0.99      1010
           4       1